In [1]:
'''
对T+CLOUD上的收发存汇总表进行二次开发，与公司以前的格式一致
此版本加零结存，将抄本中的小学生按原顺序排列
此版本增加一个按销售订单生成昨日出库的选项
此版按工厂导出的销售单格式进行处理
解决含量算法，从存货档案中导入
2024-8-12保存中间结果，将存放取文件夹中的销售订单进行汇总，作为当日出库数
'''
import os
import datetime
import easygui
import numpy as np
import pandas as pd
import openpyxl
from openpyxl.styles import Font, Border, Side, Fill, Alignment
import formatPainter
import re

dic_class = {'折扣类': '0199',
 '电商': '0106',
 '新锐': '0105',
 '复印纸': '0104',
 '锐意': '0103',
 '抄本': '0102',
 '账本': '0101',
 '电商特价': '010699',
 '定制': '010608',
 '电商抄本': '010607',
 '缝线本': '010606',
 '胶套本': '010605',
 '活页芯': '010604',
 '线环本': '010603',
 '备课本': '010602',
 '美术系列': '010601',
 '新锐订制': '010599',
 '新锐处理产品': '010598',
 '新锐防近视': '010504',
 '新锐软抄': '010503',
 '新锐胶套本': '010502',
 '新锐缝线本': '010501',
 '莱特全木浆系列': '010401',
 '锐意PP缝线活页夹': '010312',
 '锐意文稿纸': '010311',
 '锐意无线胶装': '010310',
 '锐意处理产品': '010309',
 '锐意湖北版专利作业本': '010308',
 '锐意硬抄': '010307',
 '锐意线环': '010306',
 '锐意美术系列': '010305',
 '锐意专利缝线本-黄内芯': '010304',
 '锐意软抄': '010303',
 '锐意胶套本': '010302',
 '锐意缝线本': '010301',
 '外贸订制品': '010214',
 '抄本处理产品': '010213',
 '莱特牛皮缝线本': '010212',
 '订制产品': '010211',
 '材料纸': '010210',
 '铁钉本': '010209',
 '小学生作业本': '010208',
 '拍纸簿': '010207',
 '皮面本': '010206',
 '双线环本': '010205',
 '纪念册': '010204',
 '日记本': '010203',
 '硬抄': '010202',
 '软抄': '010201',
 '经济型无碳复写单据': '010109',
 '订制账簿系列': '010108',
 '材料纸(账本）': '010107',
 '装订配件': '010106',
 '报表': '010105',
 '凭证': '010104',
 '单据': '010103',
 '账芯': '010102',
 '账簿': '010101',
 '琪丰定制': '01060806',
 '孝感思进': '01060805',
 '领创未来': '01060804',
 '佳佳定制': '01060803',
 '优本伦定制': '01060802',
 '齐达创美定制': '01060801',
 '胶装直背本': '01060704',
 '18K胶装本': '01060703',
 'B5胶装本': '01060702',
 '胶装本': '01060701',
 '卡面缝线本': '01060603',
 '牛卡缝线本': '01060602',
 '黑卡缝线本': '0106060101',
 '16K胶套本': '01060504',
 '32K胶套本': '01060503',
 'B5胶套本': '01060502',
 'A5胶套本': '01060501',
 'A4活页芯': '01060403',
 'B5活页芯': '01060402',
 'A5活页芯': '01060401',
 'PP线环本': '01060304',
 '牛卡线环本': '01060303',
 '黑卡线环本': '01060302',
 '卡面线环本': '01060301',
 '白卡软抄': '01030304',
 '特种纸软抄': '01030303',
 '32K高白软抄': '0103030202',
 '16K高白软抄': '0103030201',
 '高白软抄': '01030302',
 '16K牛皮道林软抄': '0103030103',
 '32K道林软抄': '0103030102',
 '16K道林软抄': '0103030101',
 '道林软抄': '01030301',
 '汉正街热脉小学生本': '01020828',
 '杭州科兴小学生本': '01020827',
 '人和版小学生本': '01020826',
 '山东版小学生本': '01020825',
 '佳和版小学生本': '01020824',
 '本米版小学生本': '01020823',
 '至尚版小学生本': '01020822',
 '唐山版小学生本': '01020821',
 '天津专利防近视作业本': '01020820',
 '天津东丽津南区小字本': '01020819',
 '天津版小学生本': '01020818',
 '贵州版小学生本': '01020817',
 '重庆版小学生本': '01020816',
 '成都版小学生本': '01020815',
 '汉办版小学生本': '01020814',
 '华阳版小学生本': '01020813',
 '河南版小学生本': '01020812',
 '太原版小学生本': '01020811',
 '西北版小学生本': '01020810',
 '新疆版小学生本': '01020809',
 '西安版小学生本': '01020808',
 '云南版小学生本': '01020807',
 '华北版小学生本': '01020806',
 '华东版小学生本': '01020805',
 '长沙版小学生本': '01020804',
 '广州版小学生本': '01020803',
 '安顺版小学生本': '01020802',
 '昆明版小学生本': '01020801',
 '办公硬抄': '01020203',
 '卡通硬抄': '01020202',
 '精装硬抄': '01020201',
 '精品软抄': '01020104',
 '无线胶装软抄': '01020103',
 '办公软抄': '01020102',
 '卡通软抄': '0102010102',
 '无碳复写单据': '01010303',
 '多联单据': '01010302',
 '单联单据': '01010301',
 '立信孔账芯': '01010202',
 '普通账芯': '01010201',
 '皮面账簿': '01010102',
 '纸面账簿': '01010101',
 '32K牛卡缝线本': '0106060203',
 '16K牛卡缝线本': '0106060202',
 'A4牛卡缝线本': '0106060201',
 '16K黑卡缝线本': '0106060102',
 '杨曙华定制': '01060807',
 '外贸': '0107',
 '肥猫C定制': '01060808',
 '取蓝缝线本': '01060604'}

#含量字典
content_dic = {'1052400': 24, '1052402': 24, '1052403': 96, '1052403.': 48, '1101': 60, '1103': 60, '1105': 60, '1106': 60, '1107': 60, '1109': 60, '1110': 60, '1113': 60, '1115': 60, '1116': 60, '1117': 60, '1120': 60, '1121': 60, '1122': 60, '1123': 60, '1124': 60, '1125': 60, '1126': 60, '1206': 100, '1208': 100, '1210': 100, '1225': 100, '1508废次品': 140, '1601': 220, '1602': 220, '1603': 220, '16K30页牛皮草稿本湖南': 240, '16K30页牛皮分区草稿本': 240, '16K50页牛皮草稿本湖南': 160, '16KPP账夹': 260, '16K备课本-30P': 240, '16K备课本-50Pa': 180, '16K备课本50Pt': 180, '16K废次品': 120, '16K缝线废次品': 160, '2001': 270, '2002': 270, '2003': 270, '2004': 270, '2005': 160, '2301': 270, '24K大横格本(贵州)': 960, '24K单行本（贵州）a': 960, '24K课文簿(贵州)': 960, '24K拼音写字簿(贵州)': 960, '24K数学本(贵州)': 960, '24K中方格簿(贵州)': 960, '25KPP账夹': 340, '25KPP账夹.': 340, '3001': 270, '3002': 270, '3003': 270, '3004': 270, '3007': 420, '3008': 420, '3010': 270, '3011': 270, '3013': 270, '3014': 300, '3015': 270, '3015D': 480, '3016': 270, '3016*': 480, '3017': 270, '3018': 270, '32K废次品': 200, '3333': 0, '3501': 420, '3502': 420, '3503': 420, '3504': 420, '3601': 300, '3602': 300, '3603': 270, '3605': 540, '401': 1000, '401F': 1000, '401Z': 800, '402': 1000, '402Z': 800, '404': 1000, '404F': 1000, '404Z': 800, '406': 1000, '406F': 1000, '406Z': 800, '407横': 1000, '408': 1000, '408F': 1000, '408Z': 800, '409': 1000, '409F': 1000, '409Z': 800, '413': 1000, '413Z': 800, '4444': 0, '4K160g素描纸': 60, '4K素描纸': 80, '5007': 360, '5008': 360, '5009': 360, '5014': 360, '5017': 240, '5019': 420, '502': 1000, '5027': 490, '5029': 490, '503': 1000, '5037': 420, '5039': 420, '5049': 490, '509': 1000, '509Z': 800, '5129': 420, '5139': 360, '51605401': 40, '51605701': 80, '51613401': 40, '51613701': 80, '51613711': 80, '5207': 420, '5209': 420, '5327EG': 320, '5333EG': 320, '5334EG': 320, '5337EG': 320, '5347E': 320, '5347EG': 320, '5425EG': 480, '5429EG': 480, '5433EG': 480, '5434EG': 480, '5435EG': 480, '5439EG': 480, '5443EG': 480, '5444E': 480, '5444EG': 480, '5445EG': 480, '5449EG': 480, '5501': 360, '5503': 360, '5504': 360, '5513': 240, '5514': 240, '5555': 0, '5603': 240, '5623': 280, '6001': 60, '6003': 60, '6005': 60, '602': 960, '602F': 600, '602L': 600, '602Z': 720, '603': 960, '603F': 960, '603Z': 720, '604': 960, '604F': 960, '604Z': 720, '605': 960, '605F': 960, '605Z': 720, '606': 960, '606F': 960, '606Z': 720, '6101': 40, '6102': 40, '6103': 40, '6105': 40, '6106': 40, '6107': 40, '6108': 40, '6110': 40, '6113': 40, '6115': 40, '6116': 40, '6117': 40, '6118': 40, '6120': 40, '6121': 40, '6123': 40, '6124': 40, '6125': 40, '6126': 40, '6128': 40, '6131': 60, '6133': 60, '6135': 60, '6181': 60, '6183': 60, '6185': 60, '6191': 60, '6193': 60, '6195': 60, '6206': 50, '6208': 50, '6210': 50, '6211': 70, '6213': 70, '6215': 70, '6401': 60, '6403': 60, '6405': 60, '6501': 280, '6502': 70, '6503': 100, '6504': 70, '6666': 0, '6701': 60, '6703': 60, '6705': 60, '7001': 110, '7002': 220, '7003': 110, '7005': 220, '7007': 220, '7008': 200, '7009': 90, '7010': 220, '7011': 220, '7012': 200, '7013': 200, '7015': 220, '7501': 110, '7502': 110, '7777': 0, '7777-1': 0, '8101(二孔)': 100, '8210': 90, '8211': 80, '8370': 500, '8888': 0, '8K160g素描纸': 120, '8K素描纸': 160, '9201': 20, '9202': 20, '9501': 20, '9502': 20, '9999': 0, 'A1-052399': 24, 'A1-052400': 24, 'A1-052402': 24, 'A1-052403': 48, 'A1-052405': 24, 'A1-052406': 24, 'A1-054197': 20, 'A1-054199': 24, 'A1-054428': 96, 'A470克复印纸': 8, 'A480克复印纸': 8, 'A4单线信纸': 300, 'A4双线信纸': 300, 'A4图画本（混版）a': 360, 'A5-52页书芯（箱装）': 140, 'A5-52页书芯t': 144, 'A5-92页书芯（箱装）*': 116, 'A5-92页书芯t': 100, 'A5英语本(贵州)': 800, 'A5作文本(贵州)': 800, 'ATHA460B-独角之王': 140, 'ATHA460B-格林童话': 140, 'ATHA460B-户外撒野': 140, 'ATHA460B-混装': 140, 'ATHA460B-卡通头像': 140, 'ATHA460B-连连看': 140, 'ATHA460B-萌宠派对': 140, 'ATHA460B-森林谜语': 140, 'ATHA460B-银河星工厂': 140, 'ATHA460N': 140, 'ATHA460-独角之王': 140, 'ATHA460-格林童话': 140, 'ATHA460-户外撒野': 140, 'ATHA460-混装': 140, 'ATHA460-卡通头像': 140, 'ATHA460-连连看': 140, 'ATHA460-萌宠派对': 140, 'ATHA460-森林谜语': 140, 'ATHA460-银河星工厂': 140, 'B5-152页书芯t': 50, 'B5-52页书芯（箱装）': 116, 'B5-52页书芯t': 100, 'B5-68页书芯t': 92, 'B5-92页书芯（箱装）*': 72, 'B5-92页书芯（箱装）a': 52, 'B5-92页书芯t': 68, 'B5软抄废次品': 200, 'B5图画本（混版）a': 520, 'B5图画本（男版）a': 520, 'B5图画本（女版）a': 520, 'BH101Z': 680, 'BH102Z': 680, 'BH103Z': 680, 'BH105Z（9格）': 680, 'BH106Z（9格）': 680, 'BH201Z': 360, 'BH202Z': 360, 'BH203Z': 360, 'BH207Z': 360, 'BLOCNOTEA6': 160, 'BM101Z': 400, 'BM103Z': 400, 'BM104Z': 400, 'BM105Z': 400, 'BM107Z': 400, 'BM201Z': 200, 'BM202Z': 200, 'BM203Z': 200, 'BM205Z': 200, 'BM206Z': 200, 'BM208Z': 200, 'BM210Z': 200, 'BQ1640': 280, 'BQ1640t': 240, 'BTHA460B-混装': 160, 'BTHA460B-可爱日记': 160, 'BTHA460B-快乐': 160, 'BTHA460B-漫游宇宙': 160, 'BTHA460B-美食总动员': 160, 'BTHA460B-迷失花丛': 160, 'C5029': 490, 'CD101': 1200, 'CD101NH': 540, 'CD102NH': 540, 'CD103NH': 540, 'CD104': 1200, 'CD104NH': 540, 'CD105NH': 540, 'CD106': 1200, 'CD106NH': 540, 'CD107NH': 540, 'CD108NH': 540, 'CD109NH': 540, 'CD201': 600, 'CD201NH': 280, 'CD202NH': 280, 'CD203': 600, 'CD203NH': 280, 'CD204': 600, 'CD204NH': 280, 'CD205': 600, 'CD205NH': 280, 'CD206NH': 280, 'CD207NH': 280, 'CGN1640': 280, 'CGN1640-5本': 280, 'CNF1660': 160, 'CNF1680': 120, 'CNF3260a': 280, 'CNFA480': 120, 'CQ103': 1000, 'CQ103a': 600, 'CQ105a': 600, 'CQ105Nb': 800, 'CQ106': 600, 'CQ108': 1000, 'CQ108a': 600, 'CQ1660-混装': 200, 'CQ201': 480, 'CQ302Na': 400, 'CQ305': 480, 'CQ305Na': 400, 'CQL-1660-牛卡': 200, 'CQL-1680-牛卡': 140, 'CQY-1660-牛卡': 200, 'CQY-1680牛卡': 140, 'CQZW-1660牛卡': 200, 'CQZW-1660-牛卡': 200, 'CQZW-1680牛卡': 140, 'CQZW-1680-牛卡': 140, 'CS101': 1200, 'CS102': 1200, 'CS105': 1200, 'CS107': 1200, 'CS108': 1200, 'CS110': 1200, 'CTHA460B-复古猫猫': 120, 'CTHA460B-果果联盟': 120, 'CTHA460B-欢乐时光': 120, 'CTHA460B-混装': 120, 'CTHA460B-暖暖春风': 120, 'CTHA460B-遇见敦煌': 120, 'CTHA460-混装': 120, 'DB501': 500, 'DC-1640': 200, 'DC-1640-混装': 200, 'DC-1660': 160, 'DC-1660B-乡间乐趣': 160, 'DC-1660B-小确幸': 160, 'DC-1660W-喵趣恋': 160, 'DC-1660W-随心周末': 160, 'DC-1660W-寻星记': 160, 'DC-1660Z-惬意时光': 160, 'DC-1660Z-甜密': 160, 'DC-1660-混装': 160, 'DC-1680': 100, 'DC-1680A-乐享之歌': 100, 'DC-1680A-小确幸': 100, 'DC-1680A-寻梦之旅': 100, 'DC-1680B-说文解字': 100, 'DC-1680B-小熊星座': 100, 'DC-1680-混装': 100, 'DJC1680': 100, 'DJDK100A-纷飞一夏': 384, 'DJDK100A-太空人': 384, 'DJDK100A-天台楼阁': 384, 'DJDK100B-魔法小熊': 384, 'DJDK100B-时光线': 384, 'DJDK100B-踏青': 384, 'DJDK100B-银河穿行': 384, 'DJDK100-幸运星': 384, 'DJDK90A-春日之光': 288, 'DJDK90A-花间私语': 288, 'DJDK90B吉乐兔': 288, 'DJDK90B-水果派对': 288, 'DJDK90B-一帆风顺': 288, 'DJDK90B-宇宙之歌': 288, 'DJDK90B-只此江山': 288, 'DJL100混装': 360, 'DJL16100A-且慢': 80, 'DJL16100B-浮光掠影': 80, 'DJL16100B-腹有诗书': 80, 'DJL16100B-山与川': 80, 'DJL16100C-千里江山图': 80, 'DJL16150A-一城一景': 60, 'DJL16150B-建筑说': 60, 'DJL16150B-经典之作': 60, 'DJL16150B-特点': 60, 'DJL16150B-烟雨江南': 60, 'DJL16150C-风吹麦浪': 60, 'DJL16200B-花间失落': 40, 'DJL16200B-简约生活': 40, 'DJL16200B-旅行浪漫': 40, 'DJL16200C-笔墨行云': 40, 'DJL1680B-奶茶物语': 100, 'DJL1680B-色染山川': 100, 'DJL1680B-一朵云': 100, 'DJL1680C-青春时光': 100, 'DJL1680C-野生探险': 100, 'DJL32100A-叶的思念': 120, 'DJL32100B-喵的日常': 120, 'DJL32100B-鸟语花香': 120, 'DJL32100B-只此江山': 120, 'DJL32150A-人鱼之梦': 80, 'DJL32150B-春日限定': 80, 'DJL32150B-手书': 80, 'DJL3260A-花样年华': 160, 'DJL3260B-可爱兔': 160, 'DJL3260C-晴空万里': 160, 'DJL3280A-不一样组合': 160, 'DJL3280B-看我心情': 160, 'DJL3280B-失眠星球': 160, 'DJL3280B-水果派对': 160, 'DJL3280B-印象梵高': 160, 'DJL3280C-山川古亭': 160, 'DJL3280C-油画印象': 160, 'DJL3280Y-心灵梦境': 160, 'DJL64混装': 240, 'DJL80混装': 240, 'DL-16100B-浮梦追光': 80, 'DL-16100B-威尼斯之城': 80, 'DL-16100B-宅的故事': 80, 'DL-16100-风的颜色': 80, 'DL-16100-努力学习': 80, 'DL-16100-人间浪漫': 80, 'DL-16150C-穿越时空': 60, 'DL-16150C-风的告白': 60, 'DL-16150C-青春方程': 60, 'DL-16150C-诗言': 60, 'DL-16150混装a': 60, 'DL-16200B-欢乐屋': 48, 'DL-16200B-时间记者': 48, 'DL-16200B-太空记': 48, 'DL-16200C-生活的答案': 48, 'DL-16200C-忘忧': 48, 'DL-16200C-线条空间': 48, 'DL-16200-混装': 48, 'DL-1640B-花间失落': 200, 'DL-1640C-果果爱': 200, 'DL-1660A-且慢': 160, 'DL-1660A-有米陪伴': 160, 'DL-1660B-笔刷': 160, 'DL-1660B-捕捉快乐': 160, 'DL-1660B-看我心情': 160, 'DL-1660B-烟雨江南': 160, 'DL-1660B-印象梵高': 160, 'DL-1660C-青春故事': 160, 'DL-1660C-自由不落俗': 160, 'DL-1660Z-云中许愿瓶': 160, 'DL-1660-都市恋曲': 160, 'DL-1660混装': 160, 'DL-1660-四君子Q': 160, 'DL-1680A-奔赴远方': 100, 'DL-1680B-初光': 100, 'DL-1680B-点心传说': 100, 'DL-1680B-风景日记': 100, 'DL-1680B-浮光掠影': 100, 'DL-1680B-简约生活': 100, 'DL-1680B-猫言': 100, 'DL-1680B-失眠星球': 100, 'DL-1680B-世界环游': 100, 'DL-1680B-瓦尔登湖': 100, 'DL-1680B-一朵云': 100, 'DL-1680C-冬日私语': 100, 'DL-1680C-花开季节': 100, 'DL-1680C-每日计划': 100, 'DL-1680C-天空之境': 100, 'DL-1680Z-猫空之城': 100, 'DL-1680Z-每日梦想': 100, 'DL-1680-布格子（订制）': 100, 'DL-1680-风飘飘': 100, 'DL-1680混装a': 100, 'DL-3260B-威尼斯之城': 240, 'DL-3260B-宇宙之歌': 240, 'DL-3260混装': 240, 'DL-3280Z-熊熊': 200, 'DL-3280-浮梦追光': 200, 'DL-3280-你好晚安': 200, 'DL-3280-守候': 200, 'DOUBLELINE-32P-3MM': 400, 'DR-3260-快乐相伴': 240, 'DR-3280-混装': 200, 'DRAWNBOOK22*17-16P': 400, 'DRXBA5100': 128, 'DRXBA6100': 256, 'DRXBB5100': 96, 'DRXFA5100': 128, 'DRXFA6100': 256, 'DRXFB5100': 96, 'DRXLA5100': 128, 'DRXLA6100': 256, 'DRXLB5100': 96, 'DRXYA5100': 128, 'DRXYA6100': 256, 'DRXYB5100': 96, 'DWXA5100N': 100, 'DWXA540N': 240, 'DWXA560N': 200, 'DWXA580N': 140, 'DWXB5100N': 80, 'DWXB540N': 160, 'DWXB560N': 120, 'DWXB580N': 100, 'DX100-环游星球': 480, 'DX100-请多指教': 480, 'DX100-天空之境': 480, 'DX100-一间小店': 480, 'DX100-有一束光': 480, 'DXA5100混装': 96, 'DXA6100混装': 192, 'DXA7100混装': 240, 'DY-16100A-夏日重现': 80, 'DY-16100混装': 80, 'DY-1640': 200, 'DY-1640A-熊仔': 200, 'DY-1640A-勇敢的心': 200, 'DY-1640W-加油吧': 200, 'DY-1640W-静谧时光': 200, 'DY-1640W-星': 200, 'DY-1640W-寻找你': 200, 'DY-1660A-彩虹梦': 160, 'DY-1660A-七彩光年': 160, 'DY-1660B-时光线': 160, 'DY-1660V-花信年华': 160, 'DY-1660W-时间小记': 160, 'DY-1660W-因为梦想': 160, 'DY-1660X-步履': 160, 'DY-1660-独家记忆': 160, 'DY-1660混装': 160, 'DY-1660-晴了S': 160, 'DY-1680A-守候幸福': 100, 'DY-1680A-夏日之凉': 100, 'DY-1680-B北海风光': 100, 'DY-1680B-纪录的事': 100, 'DY-1680-春景（定制）': 100, 'DY-1680混装': 100, 'DY-3260': 240, 'DY-3260X-果果': 240, 'DY-3280X-梦游': 200, 'DY-3280X-宇宙恋歌': 200, 'DY-3280-粉红的梦S': 200, 'DY-3280混装a': 200, 'DY-3280-世界那么大S': 200, 'DZW-1640-快乐值': 200, 'DZW-1640-小可爱': 200, 'DZW-1640-小蜜蜂': 200, 'DZW-1640-遥远的思念': 200, 'DZW-1660': 160, 'DZW-1660-海边游记': 160, 'DZW-1660-海边游记*': 160, 'DZW-1660-户外撒野': 160, 'DZW-1660混装（大格)': 160, 'DZW-1660-天宫阙': 160, 'DZW-1660-温柔岁月': 160, 'DZW-1660-星星陪你（大格）': 160, 'DZW-1680': 100, 'DZW-1680-春日静语': 100, 'DZW-1680-海的秘密': 100, 'DZW-1680混装': 100, 'DZW-1680-清晨时光T': 100, 'DZW-1680-时光有你': 100, 'DZW-1680-时光有你(大格）': 100, 'DZW-1680-夏天': 100, 'DZW-1680-夏天的风': 100, 'DZW-1680-熊': 100, 'DZW-1680-燕归来': 100, 'DZW-1680-阳光正好': 100, 'DZW-1680-阳光正好（大格）': 100, 'DZW-1680-一城风情': 100, 'DZW-1680-一城风情（大格）': 100, 'EFB1660NB': 200, 'EFB3260HB（黑卡）a': 320, 'EFB3260NB（电商）': 320, 'EFBA460HB': 200, 'EFBA460NB': 200, 'EFC1660K': 200, 'EFC3260N（方格）a': 320, 'EFHP1660N': 200, 'EFHP1660N(10本）': 200, 'EFHP3260N': 320, 'EFJC3260Na': 320, 'EFL1660HB': 200, 'EFL1660K': 200, 'EFL1660NB': 200, 'EFL1660NB-定制': 200, 'EFL3260HB（黑卡）a': 320, 'EFL3260NB（电商）': 320, 'EFL3260NB-定制': 320, 'EFLA460HB': 200, 'EFLA460NB': 200, 'EFLA460NB-定制': 200, 'EFLA560K-甜系a': 200, 'EFLA560K-甜系定制': 288, 'EFLA560K甜系定制混装': 288, 'EFLA560K-盐系a': 288, 'EFLA560K-盐系定制': 288, 'EFLA560K盐系定制混装': 288, 'EFLB560K-甜系': 160, 'EFLB560K-盐系': 160, 'EFLB560K-盐系定制': 160, 'EFLB560K盐系定制混装': 160, 'EFPY1660N': 200, 'EFSX1660K': 200, 'EFSX3260N（南昌）': 320, 'EFSX3260N（双竖线）a': 320, 'EFSZ3260Na': 320, 'EFTZ1660N': 200, 'EFTZ1660N（10本）': 200, 'EFTZ3260Na': 320, 'EFWG1660NB': 200, 'EFWG3260NB（电商）': 320, 'EFWGA460NB': 200, 'EFY1660HB': 200, 'EFY1660K': 200, 'EFY1660NB': 200, 'EFY3260HB': 320, 'EFY3260NB（电商）': 320, 'EFYA460HB': 200, 'EFYA460NB': 200, 'EFYB1660N（米格）': 200, 'EFYB1660N（田格）': 200, 'EFYB1660N-米格10本': 200, 'EFYB1660N-田格10本': 200, 'EFZW1660K': 200, 'EFZW3260Na': 320, 'EHBA460': 108, 'EHBA560': 220, 'EHBA580': 192, 'EHBB560': 132, 'EHBB580': 108, 'EHDA560': 220, 'EHDA580': 192, 'EHDB560': 132, 'EHDB580': 108, 'EHFA460': 108, 'EHFA560': 220, 'EHFA580': 192, 'EHFB560': 132, 'EHFB580': 108, 'EHJCA580': 192, 'EHJCB560': 132, 'EHJCB580': 108, 'EHKNA580': 192, 'EHKNB560': 132, 'EHKNB580': 108, 'EHLA460': 108, 'EHLA560': 220, 'EHLA580': 192, 'EHLB560': 132, 'EHLB580': 108, 'EHYA560': 220, 'EHYA580': 192, 'EHYB560': 132, 'EHYB580': 108, 'EJCG1630N': 240, 'EJCG1630N（分区）': 240, 'EJCG1640N': 200, 'EJCG1640N（分区）': 200, 'EJCG1660N': 120, 'EJL16150-A01': 100, 'EJL16150-B01': 60, 'EJL1680-B01': 120, 'EJL32150': 140, 'EJL32150-B01': 120, 'EJL3280': 240, 'EJL3280-B01': 160, 'EJRJ1680-A01': 160, 'EJRJ3280-B01': 160, 'EJRJ3280-C01': 160, 'EJY16150-A01': 100, 'EJY1680-A01': 160, 'EJY32150': 140, 'EJY3280': 240, 'EJZBA4128': 56, 'EJZBA5128': 120, 'EJZBB5128': 80, 'EJZLA4128': 56, 'EJZLA5128': 120, 'EJZLB5128': 80, 'EJZW16150-A01': 100, 'EJZW16150-B01': 60, 'EJZW1680-A01': 160, 'EJZW32150': 140, 'EJZW3280': 240, 'EJZWGA4128': 56, 'EJZWGA5128': 120, 'EJZWGB5128': 80, 'ETCG1630N': 240, 'ETCG1640N': 200, 'ETCG1660N': 120, 'ETHA430A': 140, 'ETHA430B': 140, 'ETHA4-草莓兔': 300, 'ETHA4-花中不知处': 300, 'ETHA4-可爱互动': 300, 'ETHA4-那年我们': 300, 'ETHB5-迪迪星球': 480, 'ETHB5-海军风': 480, 'ETHB5-和风': 480, 'ETHB5-每日限定': 480, 'ETTHA420N': 200, 'EWCG25K100': 160, 'EWCGA4100': 80, 'EWCGA5100': 160, 'EWCGB5100': 100, 'EWTHA430a-萌物派': 160, 'EWTHA430Na': 160, 'EWTHA430-混装': 160, 'EWTHA430-萌物派': 160, 'EWTHA430-梦幻独角兽': 160, 'EWTHA430-那年盛夏': 160, 'EWTHA430-小熊之歌': 160, 'EWTHA430-星球宝贝': 160, 'EX-200': 100, 'EX-200TP': 60, 'EX-288': 80, 'EX-300': 80, 'EX-300TP': 40, 'EX-400': 30, 'EX-400TP': 30, 'EX-A4-300': 40, 'EX-A4-400': 30, 'EX-A4-80': 60, 'EXBA4100P': 64, 'EXBA5100HB': 112, 'EXBA5100NB': 112, 'EXBA5100Pa': 160, 'EXBA6100HB': 224, 'EXBA6100NB': 224, 'EXBA6100P': 216, 'EXBA7100P': 480, 'EXBB5100HB': 96, 'EXBB5100NB': 96, 'EXBB5100Pa': 120, 'EXDA7100P': 480, 'EXLA4100P': 64, 'EXLA5100HB': 112, 'EXLA5100NB': 112, 'EXLA5100Pa': 160, 'EXLA580-被可爱到': 176, 'EXLA580-和风煦日': 176, 'EXLA580-精灵伙伴': 176, 'EXLA580-踏梦远行': 176, 'EXLA6100HB': 224, 'EXLA6100NB': 224, 'EXLA6100P': 216, 'EXLA7100P': 480, 'EXLB5100HB': 96, 'EXLB5100NB': 96, 'EXLB5100Pa': 120, 'EXLB580-精灵伙伴': 128, 'EXTHA430N': 120, 'EXWGA4100P': 64, 'EXWGA5100HB': 112, 'EXWGA5100NB': 112, 'EXWGA5100Pa': 160, 'EXWGA6100HB': 224, 'EXWGA6100NB': 224, 'EXWGA6100P': 216, 'EXWGA7100P': 480, 'EXWGB5100HB': 96, 'EXWGB5100NB': 96, 'EXWGB5100Pa': 120, 'EXYA5100Pa': 160, 'EXYB5100Pa': 120, 'F401NH': 600, 'F401NH-1': 600, 'F402NH': 600, 'F402NH-1': 600, 'F404NH': 600, 'F404NH-1': 600, 'F406NH': 600, 'F406NH-1': 600, 'F408NH': 600, 'F408NH-1': 600, 'F409NH': 600, 'F409NH-1': 600, 'F602NH': 480, 'F602NH-1': 480, 'F603NH': 480, 'F603NH-1': 480, 'F604NH': 480, 'F604NH-1': 480, 'F605NH': 480, 'F605NH-1': 480, 'F606NH': 480, 'F606NH-1': 480, 'F609NH': 480, 'F609NH-1': 480, 'FB503': 500, 'FDS1660': 160, 'FDS3260a': 280, 'FHT1660': 160, 'FJC1660-卡面': 160, 'FJC3260-卡面R': 280, 'FJL1660': 160, 'FKM-1660-卡面': 160, 'FKM1660-牛皮': 160, 'FKT1660-（文科）': 160, 'FL-1660B-捕捉美好': 160, 'FL-1660B-经典之作': 160, 'FL-1660-余生美好': 160, 'FL-1680B-风景日记': 120, 'FL-1680B-只此江山': 120, 'FL-1680C-中国风': 120, 'FL-3260': 280, 'FL-3260A-花满城': 280, 'FL-3260C-风吹麦浪': 280, 'FYP1660': 160, 'FZY-3260B-漫游宇宙': 280, 'FZY-3260B-水果派对': 280, 'GDX1660': 200, 'GFG1660': 200, 'GMZ1660': 200, 'GSX1660': 200, 'GTZ1660': 200, 'GZ101': 1200, 'GZ101混装': 1200, 'GZ102': 1200, 'GZ103': 1200, 'GZ104': 1200, 'GZ105': 1200, 'GZ106': 1200, 'GZ107竖': 1200, 'GZ108': 1200, 'GZ110': 1200, 'GZ111': 1200, 'GZ112': 1200, 'GZ114': 1200, 'GZ115': 1200, 'GZ116': 1200, 'GZ118': 1200, 'GZ201': 600, 'GZ201混装': 600, 'GZ202': 600, 'GZ203': 600, 'GZ204': 600, 'GZ205': 600, 'H1205': 100, 'H1206': 100, 'H1208': 100, 'H1225': 100, 'HB101C': 950, 'HB101C彩': 950, 'HB103': 900, 'HB103C': 950, 'HB104C': 950, 'HB104C彩': 950, 'HB105C': 950, 'HB105C彩': 950, 'HB106': 900, 'HB106C': 950, 'HB106C彩': 950, 'HB107C': 950, 'HB108A': 950, 'HB108C': 950, 'HB110C': 950, 'HB110C彩': 950, 'HB201C': 400, 'HB202C': 400, 'HB203C': 400, 'HB203C彩': 400, 'HB204C': 400, 'HB204C彩': 400, 'HB205C': 400, 'HB205C彩': 400, 'HB206C': 400, 'HB206C彩': 400, 'HB208C': 400, 'HB208C彩': 400, 'HB209C': 400, 'HB209C彩': 400, 'HB210C': 400, 'HB211C': 400, 'HB212C': 400, 'HB213C': 400, 'HB214C': 400, 'HB6107': 40, 'HB6108': 40, 'HB6115': 40, 'HB6116': 40, 'HB6124': 40, 'HB6126': 40, 'HC-A4-100PL（137）': 60, 'HC-A4-200PL': 40, 'HC-A4-200SQ': 40, 'HCA4-288P': 30, 'HC-A4-300': 24, 'HC-A4-300PL': 30, 'HC-A4-300SQ': 30, 'HC-A4-400': 20, 'HC-A4-400PL': 20, 'HC-A4-400SQ': 20, 'HC-A4-500PL': 20, 'HC-R5-288P': 36, 'HC-R5-300（202）': 36, 'HC-R5-400（205）': 24, 'HD201a': 480, 'HD203a': 480, 'HNB-50029': 20, 'HNB-50200PL': 40, 'HNB-50300PL*': 32, 'HNB-50500PL': 20, 'HNC1660N（单线）': 160, 'HNCA560N（方格）': 300, 'HNCLA560N（双线）': 300, 'HNLS1660N': 160, 'HNPYA560N': 300, 'HNSH1660N': 160, 'HNSX1660N': 160, 'HNSXA560N': 300, 'HNSZA560N': 300, 'HNTZA560N': 300, 'HNWL1660N': 160, 'HNY1660N': 160, 'HNYA560N': 300, 'HNZW1660N（300格）': 160, 'HNZW1660N（400格）': 160, 'HNZWA560N': 300, 'HWC1840N': 280, 'HWHP1840N': 280, 'HWLZ1840N': 280, 'HWPX1840N': 280, 'HWSX1840N': 280, 'HWSZ1840N': 280, 'HWTZ1840N': 280, 'HWY1840N': 280, 'HWZW1840N': 280, 'HY101': 960, 'HY102': 960, 'HY107': 960, 'HY111': 960, 'HY113': 960, 'HY117': 960, 'HY602混装a': 800, 'HZ101': 630, 'HZ102': 630, 'HZ103': 630, 'HZ104': 630, 'HZ105': 630, 'HZ106': 630, 'HZ107': 630, 'HZ302': 320, 'HZJX3280': 160, 'J1601': 270, 'J1602': 270, 'J1603': 270, 'J1605': 320, 'J1606': 320, 'J1607': 320, 'J1608': 320, 'JFC1650N（方格）': 240, 'JFC1660N（单行）': 200, 'JFC1660N（方格）': 200, 'JFC1660N（双行）': 200, 'JFC3260N': 320, 'JFCT1660N': 200, 'JFDL1660N': 200, 'JFDS1660N': 200, 'JFDS3260N': 320, 'JFHX1660N': 200, 'JFJA1650N': 240, 'JFJA1660N': 200, 'JFJC1660N': 200, 'JFJC3260N': 320, 'JFL1660N': 200, 'JFL3260N': 320, 'JFLS1660N': 200, 'JFRJ1660N': 200, 'JFRJ3260N': 320, 'JFSW1660N': 200, 'JFSX1650N（单行）': 240, 'JFSX1650N（双竖线）': 240, 'JFSX1660N（单行）': 200, 'JFSX1660N（双竖线）': 200, 'JFSX3260N': 320, 'JFSZ1650N': 240, 'JFSZ1660N': 200, 'JFSZ3260N': 320, 'JFTZ1660N': 200, 'JFWL1660N': 200, 'JFY1650N': 240, 'JFY1660N': 200, 'JFY3260N': 320, 'JFYD1660N': 200, 'JFZW1650N（300格）': 240, 'JFZW1660N（300格）': 200, 'JFZW1660N（400格）': 200, 'JFZW3260N': 320, 'JFZY3260N': 320, 'JFZZ1660N': 200, 'JJTH101': 450, 'JJTH201': 190, 'JSCG16100': 100, 'JSCG1630': 240, 'JSCG1630（分区）': 240, 'JSCG1640': 200, 'JSCG1640（分区）': 200, 'JSCG1660': 120, 'JT-B5数学': 200, 'JT-B5英语': 200, 'JT-B5语文': 200, 'JT-B5语文 箱装': 200, 'KM101': 1200, 'KM101混装': 1200, 'KM102': 1200, 'KM103': 1200, 'KM103N': 1100, 'KM103Z': 800, 'KM104': 1200, 'KM104N': 1100, 'KM106': 1200, 'KM107': 1200, 'KM108': 1200, 'KM108N': 1100, 'KM201': 880, 'KM201N': 880, 'KM202': 880, 'KM202Z': 600, 'KM203': 880, 'KM203N': 880, 'KM302N': 560, 'KM302Z': 400, 'KM312N': 400, 'KM313N': 400, 'KM315N': 400, 'KM316N': 400, 'KM401A': 1040, 'KM401A-1': 560, 'KM402（米黄）': 1040, 'KM402A': 1040, 'KM402A-1': 560, 'L1106': 60, 'L1109': 60, 'L1110': 60, 'L1113': 60, 'L1117': 60, 'L1123': 60, 'L1125': 60, 'L1126': 60, 'L16KPP': 260, 'L6703': 60, 'L6705': 60, 'MB504': 500, 'MFC1680RM-F16801': 120, 'MFC3260RM-F32601': 320, 'MFDS1660RM-F16606': 160, 'MFDS3260RM-F32606': 320, 'MFHP1660RM-F16608': 160, 'MFHP3260RM-F32608': 320, 'MFJC3260RM-F32607': 320, 'MFL1680RM-F16804': 120, 'MFRJ3260RM-F32609': 320, 'MFSX1680RM-F16802': 120, 'MFSX3260RM-F32602': 320, 'MFY1680RM-F16803': 120, 'MFY3260RM-F32603': 320, 'MFZW1660RM-F16605': 160, 'MFZW1680RM-F16805': 120, 'MFZW3260RM-F32605': 320, 'MJC1640RM-J16401': 200, 'MJC1660RM-J16601': 160, 'MJC1680RM-J16801': 100, 'MJCP3240RM-J32406': 480, 'MJDS3240RM-J32402': 480, 'MJHP1640RM-J16408': 200, 'MJHP3240RM-J32408': 480, 'MJL1640RM-J16404': 200, 'MJL1660RM-J16604': 160, 'MJL3240RM-J32404': 480, 'MJRJ3240RM-J32409': 480, 'MJSX1640RM-J16402': 200, 'MJSX1660RM-J16602': 160, 'MJSX1680RM-J16802': 100, 'MJTZ3240RM-J32405': 480, 'MJXZ3240RM-J32401': 480, 'MJY1640RM-J16403': 200, 'MJY1660RM-J16603': 160, 'MJY1680RM-J16803': 100, 'MJY3240RM-J32403': 480, 'MJZW1640RM-J16405': 200, 'MJZW1660RM-J16605': 160, 'MJZW1680RM-J16805': 100, 'N0002': 400, 'N0003': 420, 'N0007': 300, 'N0311': 360, 'N0321': 360, 'N0331': 360, 'N0332': 320, 'N0341': 360, 'N0342': 320, 'N0376-意景': 360, 'N0422': 440, 'N0422-5本': 440, 'N0423': 320, 'N0424': 280, 'N0424废次品': 160, 'N0425': 220, 'N0430': 180, 'N0481': 320, 'N0482': 240, 'N0483': 180, 'N0484': 140, 'N0511': 320, 'N0516办': 240, 'N0517': 160, 'N0518': 120, 'N0519': 120, 'N0520': 100, 'N0521': 320, 'N0526': 240, 'N0527': 160, 'N0528': 120, 'N0529': 120, 'N0530': 100, 'N0531': 320, 'N0541': 320, 'N0565': 130, 'N0566': 100, 'N0568': 80, 'N0569': 60, 'N0574': 240, 'N0576': 240, 'N0576废次品': 160, 'N0577': 160, 'N0578': 120, 'N0578废次品': 90, 'N0581': 200, 'N0582': 160, 'N0583': 120, 'N0583t': 120, 'N0584': 100, 'N0584t': 100, 'N0591': 120, 'N0595': 240, 'N0596': 160, 'N0597': 120, 'N0599': 100, 'N0651': 144, 'N0652': 120, 'N0653': 108, 'N0654': 84, 'N0661': 144, 'N0662': 120, 'N0663': 108, 'N0664': 84, 'N06641': 240, 'N06642': 160, 'N06643': 120, 'N06644': 100, 'N0667': 140, 'N0668': 140, 'N0669': 144, 'N0670': 480, 'N0671': 320, 'N0672': 240, 'N0673': 180, 'N0674': 140, 'N0681': 320, 'N0682': 240, 'N0683': 180, 'N0684': 140, 'N0694': 280, 'N0709': 60, 'N0737': 240, 'N0738': 240, 'N0742': 240, 'N0765': 130, 'N0766': 100, 'N0767': 72, 'N0780': 200, 'N0780（电商）a': 200, 'N0780（南昌）': 200, 'N0781（单线）': 200, 'N0781（方格）': 200, 'N0781（方格）南昌': 200, 'N0781（双行）': 200, 'N07810': 200, 'N0781方格（电商）a': 200, 'N0781方格电商t': 200, 'N0782（单线）': 200, 'N0782（单线）南昌': 200, 'N0782（双竖线）': 200, 'N0782（双竖线）南昌': 200, 'N0782双竖线（电商）a': 200, 'N0782双竖线电商t': 200, 'N0783': 200, 'N0783（电商）a': 200, 'N0783（南昌）': 200, 'N0783电商t': 200, 'N0784（300格）': 200, 'N0784（300格）电商a': 200, 'N0784（300格）南昌': 200, 'N0784（400格）': 200, 'N0784（400格）电商a': 200, 'N0784（400格）南昌': 200, 'N0784-300格电商t': 200, 'N0785': 200, 'N0785(5本）': 200, 'N0785（纯净版）': 200, 'N0785（纯净版）电商': 200, 'N0785（电商）a': 200, 'N0785（南昌）': 200, 'N0785-笔记': 200, 'N0785电商t': 200, 'N0786': 200, 'N0786（电商）a': 200, 'N0786（南昌）': 200, 'N0787': 200, 'N0787（电商）a': 200, 'N0787（南昌）': 200, 'N0788': 200, 'N0789': 200, 'N0791': 320, 'N0791（电商）': 320, 'N0792': 320, 'N0792（电商）': 320, 'N0792(南昌）': 320, 'N0793': 320, 'N0793（电商）': 320, 'N0793（南昌）': 320, 'N0794': 320, 'N0794（电商）': 320, 'N0794（南昌）': 320, 'N0795': 320, 'N0795（纯净版）': 320, 'N0795（电商）': 320, 'N0795（南昌）': 320, 'N0796': 320, 'N0796（电商）': 320, 'N0797': 320, 'N0797（电商）': 320, 'N0797（南昌）': 320, 'N0798': 320, 'N0811': 640, 'N0812': 480, 'N0813': 360, 'N0814': 320, 'N0815': 160, 'N0816': 480, 'N0817': 480, 'N0821': 560, 'N0821废次品': 360, 'N0822': 440, 'N0822R': 440, 'N0822废次品': 280, 'N0823': 320, 'N0823R': 320, 'N0823废次品': 240, 'N0824R': 240, 'N0824废次品': 160, 'N0825': 220, 'N0825R': 200, 'N0826': 180, 'N0826S-定制': 180, 'N0826废次品': 130, 'N0831': 160, 'N0835': 640, 'N0836': 360, 'N0837': 320, 'N0838': 640, 'N0839': 360, 'N0840': 320, 'NCF3260NH': 280, 'NFBJ1660NH': 160, 'NFBJ1660NH-1': 160, 'NFC-1660': 160, 'NFC1660NH(单行）': 160, 'NFC1660NH（方格）': 160, 'NFC1660NH（方格）-1': 160, 'NFC1660NH(双行）': 160, 'NFC-3260': 280, 'NFC3260NH': 280, 'NFC3260NH-1': 280, 'NFDS-1660': 160, 'NFDS1660NH': 160, 'NFDS1660NH-1': 160, 'NFDS-3260': 280, 'NFDS3260NH': 280, 'NFDS3260NH-1': 280, 'NFHP1660NH': 160, 'NFHP1660NH-1': 160, 'NFHP3260NH': 280, 'NFHP3260NH-1': 280, 'NFJC-1660': 160, 'NFJC1660NH': 160, 'NFJC1660NH-1': 160, 'NFJC-3260': 280, 'NFL-1660': 160, 'NFL1660NH': 160, 'NFL1660NH-1': 160, 'NFL-3260': 280, 'NFL3260NH': 280, 'NFL3260NH-1': 280, 'NFL-A460': 144, 'NFPH1660NH': 160, 'NFRJ1660NH': 160, 'NFRJ-3260': 280, 'NFRJ3260NH': 280, 'NFRJ3260NH-1': 280, 'NFSX-1660': 160, 'NFSX1660NH(单行）': 160, 'NFSX1660NH（双竖线）': 160, 'NFSX1660NH（双竖线）-1': 160, 'NFSX-3260': 280, 'NFSX3260NH（双竖线）': 280, 'NFSX3260NH（双竖线）-1': 280, 'NFSZ1660NH': 160, 'NFSZ1660NH-1': 160, 'NFXK1660NH': 160, 'NFY-1660': 160, 'NFY1660NH（双色）': 160, 'NFY1660NH（双色）-1': 160, 'NFY-3260': 280, 'NFY3260NH': 280, 'NFY3260NH-1': 280, 'NFY-A460': 144, 'NFZW-1660（300格）': 160, 'NFZW-1660（400格）': 160, 'NFZW1660NH（300格）': 160, 'NFZW1660NH（300格）-1': 160, 'NFZW1660NH（400格）': 160, 'NFZW1660NH（400格）-1': 160, 'NFZW3260NH': 280, 'NFZW-A460': 144, 'NFZY-3260': 280, 'NFZY3260NH': 280, 'NFZY3260NH-1': 280, 'NK101': 550, 'NK102': 550, 'NK103': 550, 'NK104': 550, 'NK105': 550, 'NK106': 550, 'NK107': 550, 'NK109-4': 550, 'NK201': 480, 'NK202': 480, 'NK301': 360, 'PJC1660': 160, 'QCY-1660牛卡': 200, 'QLX1660N': 200, 'QLX1660Nt': 200, 'QS-6604H': 32, 'QSX1660N': 200, 'QYW1660N': 200, 'QYY1660N': 200, 'QZW1660N': 200, 'RC-1660': 200, 'RC-1660-E01': 200, 'RC-1660NZ': 200, 'RC-1680NZ': 140, 'RFB-1660N': 200, 'RFBJ-A460N': 140, 'RFBJA460N原木定制t': 140, 'RFC-1660N': 200, 'RFC1660N（方格）原木定制': 200, 'RFC1660N(方格）原木定制t': 200, 'RFC1660N（双线）原木定制': 200, 'RFC1660N(双线）原木定制t': 200, 'RFC-1660NZ': 200, 'RFC-3260N': 320, 'RFC-3260NZ': 320, 'RFC3260N原木定制t': 320, 'RFDF1660N原木定制': 200, 'RFDF1660N原木定制t': 200, 'RFDH-1660N': 200, 'RFDH1660N原木定制t': 200, 'RFDS-1660N': 200, 'RFDS-1660NZ': 200, 'RFDS1660N原木定制t': 200, 'RFDS-3260N': 320, 'RFDS-3260NZ': 320, 'RFHP-1660N': 200, 'RFHP-1660NZ': 200, 'RFHP1660N原木定制t': 200, 'RFHP-3260N': 320, 'RFHP-3260NZ': 320, 'RFHP3260N原木定制t': 320, 'RFJC-1660N': 200, 'RFJC-1660NZ': 200, 'RFJC1660N原木定制t': 200, 'RFJC-3260N': 320, 'RFJC-3260NZ': 320, 'RFKN-1660N': 200, 'RFKN1660N原木定制t': 200, 'RFKW1660N原木定制t': 200, 'RFL-16100-G03': 160, 'RFL-16100-H01': 160, 'RFL-1660-G03': 200, 'RFL-1660-H01': 200, 'RFL-1660-H02': 200, 'RFL-1660N': 200, 'RFL-1660NZ': 200, 'RFL1660N原木定制': 200, 'RFL1660N原木定制t': 200, 'RFL-32100-G01': 240, 'RFL-32100-G03': 240, 'RFL-32100-H01': 240, 'RFL-3260-G03': 320, 'RFL-3260-H01': 320, 'RFL-3260-H02': 320, 'RFL-3260N': 320, 'RFL-3260NZ': 320, 'RFL-A460N': 140, 'RFPY1660N原木定制': 200, 'RFPY3260N原木定制': 320, 'RFRJ-1660N': 200, 'RFRJ-3260N': 320, 'RFRJ-3260NZ': 320, 'RFSH-1660N': 200, 'RFSH1660N原木定制t': 200, 'RFSX-1660N': 200, 'RFSX-1660NZ': 200, 'RFSX1660N原木定制t': 200, 'RFSX-3260N': 320, 'RFSX-3260NZ': 320, 'RFSX3260N原木定制': 320, 'RFSX3260N原木定制t': 320, 'RFSX-A460N': 140, 'RFSXA460N原木定制t': 140, 'RFSZ-1660N': 200, 'RFSZ-1660NZ': 200, 'RFSZ1660N原木定制t': 200, 'RFTZ-1660N': 200, 'RFTZ1660N原木定制t': 200, 'RFTZ-3260N': 320, 'RFTZ3260N原木定制t': 320, 'RFY-1660-D01': 200, 'RFY-1660-G01': 200, 'RFY-1660N': 200, 'RFY-1660NZ': 200, 'RFY1660N原木定制': 200, 'RFY1660N原木定制t': 200, 'RFY-3260N': 320, 'RFY-3260NZ': 320, 'RFY3260N原木定制': 320, 'RFY3260N原木定制t': 320, 'RFY-A460N': 140, 'RFYA460N原木定制t': 140, 'RFYB-1660N': 200, 'RFYB-1660N（田字格）': 200, 'RFYD-1660N': 200, 'RFYD1660N原木定制t': 200, 'RFYP-1660N': 200, 'RFZF1660N原木定制': 200, 'RFZF1660N原木定制t': 200, 'RFZJ-1660N': 200, 'RFZJ1660N原木定制t': 200, 'RFZW-1660N（300格）': 200, 'RFZW-1660N（320格）': 200, 'RFZW-1660N（400格）': 200, 'RFZW1660N-300格原木定制t': 200, 'RFZW1660N-320格原木定制t': 200, 'RFZW-1660NZ': 200, 'RFZW-3260N': 320, 'RFZW-3260NZ': 320, 'RFZW-A460N（500格）': 140, 'RFZWA460N原木定制t': 140, 'RFZY-3260N': 320, 'RFZY-3260NZ': 320, 'RG-300P': 28, 'RJDL-16150N': 80, 'RJHX-16150N': 80, 'RJL-16100-E02': 100, 'RJL-16100-G01': 100, 'RJL-16100-G03': 100, 'RJL-16100-H01': 100, 'RJL-16100-H02': 100, 'RJL-16150-G01': 80, 'RJL-16150-G02': 80, 'RJL-16150-G03': 80, 'RJL-16150-H01': 80, 'RJL-16200-D01': 60, 'RJL-16200-F03': 60, 'RJL-16200-G01': 60, 'RJL-16200-G02': 60, 'RJL-1680-G01': 120, 'RJL-1680-G02': 120, 'RJL-1680-G03': 120, 'RJL-1680-G04': 120, 'RJL-1680-H03': 120, 'RJL-32100-D02': 140, 'RJL-32100-G01': 140, 'RJL-32100-G02': 140, 'RJL-32100-H01': 140, 'RJL-32100-H02': 140, 'RJL-32150-G01': 100, 'RJL-32150-G02': 100, 'RJL-32150-G03': 100, 'RJL-32150-H01': 100, 'RJL-32150-H02': 100, 'RJL-32150-H03': 100, 'RJL-3280-F03': 160, 'RJL-3280-G01': 160, 'RJL-3280-G02': 160, 'RJL-3280-G03': 160, 'RJL-3280-G04': 160, 'RJL-3280-H01': 160, 'RJL-3280-H02': 160, 'RJL-3280-H03': 160, 'RJL-3280-H04': 160, 'RJLS-16150N': 80, 'RJRJ-1680-E02': 120, 'RJRJ-1680-F01': 120, 'RJRJ-1680-G01': 120, 'RJRJ-3280-G01': 160, 'RJRJ-3280-G02': 160, 'RJRJ-3280-G03': 160, 'RJRJ-3280-H01': 160, 'RJSW-16150N': 80, 'RJY-16150-G01': 80, 'RJY-1680-F01': 120, 'RJY-1680-F02': 120, 'RJY-1680-G01': 120, 'RJY-3280-G01': 160, 'RJZW-1680-E01': 120, 'RJZW-1680-H01': 120, 'RJZW-1680-H02': 120, 'RJZZ-16150N': 80, 'RL-16100': 100, 'RL-16100-G02': 100, 'RL-16100-H01': 100, 'RL-16150-G02': 80, 'RL-16150-H01': 80, 'RL-16150-H02': 80, 'RL-1660-F01': 200, 'RL-1660-G01': 200, 'RL-1660-G02': 200, 'RL-1660-H01': 200, 'RL-1660-H02': 200, 'RL-1660NZ': 200, 'RL-1660-卡面': 200, 'RL-1660-牛卡': 200, 'RL-1680-F01': 140, 'RL-1680-F02': 140, 'RL-1680-G02': 140, 'RL-1680-G03': 140, 'RL-1680-H01': 140, 'RL-1680-H02': 140, 'RL-1680-H03': 140, 'RL-1680-H04': 140, 'RL-1680-混装': 140, 'RL-1680-牛卡': 140, 'RL-32150': 240, 'RL-32150-G01': 240, 'RL-32150-H01': 240, 'RL-32150-H02': 240, 'RL-3280-G01': 280, 'RM401': 700, 'RM402': 700, 'RM404': 700, 'RM406': 700, 'RM408': 700, 'RM409': 700, 'RM602': 520, 'RM603': 520, 'RM604': 520, 'RM605': 520, 'RM606': 520, 'RM609': 520, 'RM-FA4602': 120, 'RY-1660NZ': 200, 'RY-32150': 240, 'RZW-1660': 200, 'RZW-1660NZ': 200, 'RZW-1660混装': 200, 'RZW-1680NZ': 140, 'SB502': 500, 'SEB-100': 200, 'SEB-13300': 40, 'SEB-14400': 30, 'SEB-22160*': 120, 'SEB-300': 80, 'SEB-A4-200': 60, 'SEB-A4-400': 32, 'SM1640C-冬日私语': 88, 'SM1640C-梵高的画': 88, 'SM1640C-海城之光': 88, 'SM1640C-混装': 88, 'SM1640C-连连看': 88, 'SM1640C-牛卡': 88, 'SM1640S-浮光探影': 88, 'SM1640S-魔法小熊': 88, 'SM1640S-天台楼阁': 88, 'SM1640S-我们的故事': 88, 'SM1660HB': 30, 'SM1660N': 30, 'SM1660N定制': 30, 'SM8K40S-黄昏美': 44, 'SM8K40S-牛卡': 44, 'SM8K40S-世外之源': 44, 'SM8K40S-夏日游记': 44, 'SM8K40S-只此江山': 44, 'SM8K60HB': 20, 'SM8K60N': 20, 'SM8K60N定制': 20, 'SMA340C-花之旅': 30, 'SMA340C-混装': 30, 'SMA340C-袅袅翠谷': 30, 'SMA340C-牛卡': 30, 'SMA340C-童话世界': 30, 'SMA340S-风起时': 30, 'SMA340S-混装': 30, 'SMA340S-寂静': 30, 'SMA340S-思忆随风': 30, 'SMA340S-一个人的世界': 30, 'SMA340S-一些可爱': 30, 'SMA340S-宇宙之歌': 30, 'SMA340T-墨韵芳华': 30, 'SMA340T-星空的梦': 30, 'SMA340次品': 30, 'SMA340-亿江南': 30, 'SMA340-织羽梦': 30, 'SMA440C-纷飞一夏': 60, 'SMA440C-风的颜色': 60, 'SMA440C-回忆旧事': 60, 'SMA440C-混装': 60, 'SMA440C-街边小巷': 60, 'SMA440C-牛卡': 60, 'SMA440C-日常循环': 60, 'SMA440C-世界环游': 60, 'SMA440C-银河穿行': 60, 'SMA440C-云舒霞卷': 60, 'SMA440S-春之物语': 60, 'SMA440S-混装': 60, 'SMA440S-水果派对': 60, 'SMA440S-野生探险': 60, 'SMA460HB': 30, 'SMA460N': 30, 'SXA450': 80, 'SXA450-城中往事': 80, 'SXA450-猫儿家园': 80, 'SXA450-品味生活': 80, 'SXA450-森林乐队': 80, 'SXA450-小熊故事': 80, 'SXA450-志在远方': 80, 'SXB550': 88, 'SXB550次品': 88, 'SXB550-小花园': 88, 'SXB550-袖时光': 88, 'SXB550-阳光海岸': 88, 'TB505': 500, 'TG78系列混装': 200, 'TG78系列混装5本': 200, 'THA460-都市生活': 140, 'THA460-连连看': 140, 'THA460-萌宠派对': 140, 'THWA460-果子乐园': 120, 'THWA460-满载快乐': 120, 'THWA460-软萌甜心': 120, 'THWA460-自由翱翔': 120, 'TJ101': 780, 'TJ101a': 760, 'TJ101彩': 780, 'TJ102': 760, 'TJ102彩': 760, 'TJ103': 760, 'TJ103彩': 760, 'TJ104': 760, 'TJ105': 760, 'TJ105彩': 760, 'TJ106': 760, 'TJ107彩': 760, 'TJ108': 760, 'TJ108彩': 760, 'TJ109彩': 760, 'TJ111彩': 760, 'TJ112彩': 760, 'TJ113': 760, 'TJ113彩': 760, 'TJ114彩': 760, 'TJ201彩': 600, 'TJ202彩': 600, 'TJ203彩': 600, 'TJ205': 600, 'TJ205彩': 600, 'TJ206': 600, 'TJ206彩': 600, 'TJ301': 280, 'TJ301彩': 280, 'TJ302': 280, 'TJ302彩': 280, 'TJ303': 280, 'TJ303彩': 280, 'TJ304': 280, 'TJ304彩': 280, 'TJ305': 280, 'TJ305彩': 280, 'TJ306': 280, 'TJ306彩': 280, 'TJ307彩': 280, 'TJ308': 280, 'TJ309': 280, 'TJ309a': 280, 'TJ309B彩': 280, 'TJ309彩': 280, 'TJ310': 280, 'TJ311': 280, 'TJ311彩': 280, 'TJ401': 240, 'TJ402': 240, 'TJ403': 240, 'TOP-1': 96, 'TOP-2': 96, 'TOP-3': 96, 'TOP-4': 144, 'TOP-5': 144, 'TOP-6': 144, 'TS101Z': 400, 'TS102Z': 400, 'TS103Z': 400, 'TS104Z': 400, 'TS108Z': 400, 'TS109Z': 400, 'TS301Z': 200, 'TS302Z': 200, 'TS303Z': 200, 'TY201': 400, 'TY203': 400, 'TY204': 400, 'TY205': 400, 'TY208': 400, 'WC1680（方格）': 100, 'WJCA580': 200, 'WL1680': 100, 'WY1680': 100, 'WZW1680（300格）': 100, 'XA101': 560, 'XA101NH': 450, 'XA102': 560, 'XA102NH': 450, 'XA103': 560, 'XA103NH': 450, 'XA104': 560, 'XA104NH': 450, 'XA105': 560, 'XA105NH': 450, 'XA106': 560, 'XA201': 280, 'XA201NH': 280, 'XA202': 280, 'XA203': 280, 'XA203NH': 280, 'XA204': 280, 'XA204NH': 280, 'XA205': 280, 'XA205NH': 280, 'XA206': 280, 'XA206NH': 280, 'XA207': 280, 'XA207NH': 280, 'XA208NH': 280, 'XJ101': 900, 'XJ102': 900, 'XJ103': 900, 'XJ104': 900, 'XJ105': 900, 'XJ106': 900, 'XJ201': 800, 'XJ202': 800, 'XJ204': 800, 'XJ205': 800, 'XJ206': 800, 'XJ207': 800, 'XJ208': 800, 'XJ209': 800, 'XJ303': 480, 'XJ304': 480, 'XJ305': 480, 'XJ401S': 280, 'XJ402S': 280, 'XJ403S': 280, 'XJ404S': 280, 'XJ405S': 280, 'XJ406': 280, 'XJ407': 280, 'XJ408': 280, 'XJ409': 280, 'XJ410': 280, 'XJ411': 280, 'XJ412': 280, 'XJ413': 280, 'XJ414': 280, 'XJ415': 280, 'XJ501': 440, 'XJ502': 440, 'XJ502L': 440, 'XJ503': 440, 'XJ504': 440, 'XJ505': 440, 'XJ505L': 440, 'XJ506': 440, 'XJ507': 440, 'XJ508': 440, 'XJ509': 440, 'XJ510': 440, 'XQ101': 550, 'XQ102': 550, 'XQ103': 550, 'XQ104': 550, 'XQ105': 550, 'XQ106': 550, 'XQ107': 550, 'XQ108': 550, 'XQ109': 550, 'XQ201': 480, 'XQ202': 480, 'XQ301': 360, 'YN104': 1400, 'YN106': 1400, 'YN107': 1400, 'YN201': 960, 'YN302': 800, 'YN304': 800, 'YN306': 800, 'YN307': 800, 'YS定制（混装）': 200, 'YTL1640': 320, 'YTL1660': 200, 'YTL1680': 140, 'YTY1640': 320, 'YTY1660': 200, 'YTY1680': 140, 'YTZW1640': 320, 'YTZW1660': 200, 'YTZW1680': 140, 'YW101NH': 500, 'YW102NH': 500, 'YW103NH': 500, 'YW104NH': 500, 'YW105NH': 500, 'YW107NH': 500, 'YW108NH': 500, 'YW109NH': 500, 'YW301NH': 200, 'YW302NH': 200, 'YW303NH': 200, 'YW305NH': 200, 'YW309NH': 200, 'ZS7756': 240, '存料卡': 5400, '固定资产明细卡': 3000, '护眼缝线本样品': 145, '教案本-100': 100, '教案本-80a': 160, '美国A6铁订本(枫叶)*': 36, '美国A6铁订本*': 36, '南昌2001': 270, '瑞典A460g方格': 40, '瑞典A460g线条': 40, '瑞典A470g线条': 40, '商品卡': 5400, '账钉': 100}


lst = ['仓库编码',
       '仓库',
       '存货分类 (1级)',
       '存货分类 (2级)',
       '存货分类 (3级)',
       '存货分类 (4级)',
       '存货分类 (5级)',
       '存货编码',
       '存货',
       '存货代码',
       '数量(主单位)',
       '平均单价',
       '金额',
       '数量(辅单位)',
       '计量单位组合',
       '数量(主单位).1',
       '平均单价.1',
       '金额.1',
       '数量(辅单位).1',
       '计量单位组合.1',
       '数量(主单位).2',
       '平均单价.2',
       '金额.2',
       '数量(辅单位).2',
       '计量单位组合.2',
       '数量(主单位).3',
       '平均单价.3',
       '金额.3',
       '辅数量',
       '计量单位组合.3']
lst2 = [
    'store',
    'num',
    'class01',
    'class02',
    'class03',
    'class04',
    'class05',
    'code',
    'stock',
    'content',
    'begin_ben',
    '平均单价',
    '金额',
    'begin_jian',
    '计量单位组合',
    'ruku_ben',
    '平均单价.1',
    '金额.1',
    'ruku_jian',
    '计量单位组合.1',
    'chuku_ben',
    '平均单价.2',
    '金额.2',
    'chuku_jian',
    '计量单位组合.2',
    'end_ben',
    '平均单价.3',
    '金额.3',
    'end_jian',
    '计量单位组合.3']

lst3 = [
    'class02',
    'class05',
    'code',
    'stock',
    'content',
    'begin_ben',
    'begin_jian',
    'ruku_ben',
    'ruku_jian',
    'chuku_ben',
    'chuku_jian',
    'end_ben',
    'end_jian',
]

lst_yeterday_ruku = ['class02',
                     'class05',
                     'code',
                     'stock',
                     'begin_ben',
                     'begin_jian',
                     'ruku_ben',
                     'ruku_jian',
                     ]
lst_yeterday_ruku1 = ['class02',
                      'class05',
                      'code',
                      'stock',
                      'yesterday_ben',
                      'yesterday_jian',
                      'yesterday_ruku_ben',
                      'yesterday_ruku_jian',
                      ]

lst01_yesterday = [
    'class02',
    'class05',
    'code',
    'stock',
    'begin_ben',
    'begin_jian',
    'ruku_ben',
    'ruku_jian',
    'chuku_ben',
    'chuku_jian',
]
lst02_yesterday = [
    'class02',
    'class05',
    'code',
    'stock',
    'yesterday_ben',
    'yesterday_jian',
    'yesterday_ruku_ben',
    'yesterday_ruku_jian',
    'yesterday_chuku_ben',
    'yesterday_chuku_jian',

]
lst_merge0 = ['class02',
              'class05',
              'code',
              'stock',
              'content',
              'begin_ben',
              'begin_jian',
              'ruku_ben',
              'ruku_jian',
              'chuku_ben',
              'chuku_jian',
              'end_ben',
              'end_jian',
              'yesterday_ben',
              'yesterday_jian',
              'yesterday_ruku_ben',
              'yesterday_ruku_jian',
              'yesterday_chuku_ben',
              'yesterday_chuku_jian']

lst_merge = ['class02', 'class05', 'code', 'stock',
             'content',
             'begin_jian', 'yesterday_jian',
             'yesterday_ruku_jian', 'ruku_jian',
             'yesterday_chuku_jian', 'chuku_jian',
             'end_jian']

lst_result0 = ['class05', 'code', 'stock',
               'content',
               'begin_jian', 'yesterday_jian',
               'yesterday_ruku_jian', 'ruku_jian',
               'yesterday_chuku_jian', 'chuku_jian',
               'end_jian']

lst_result = ['类别', '货号', '品名',
              '含量',
              '月初', '上日',
              '本日入库', '入库累计',
              '本日出库', '出库累计',
              '结余']


def chuli(fname, store_num):
    df = pd.read_excel(fname, skiprows=7)
    df = df.iloc[:, 1:]
    df.columns = lst
    df.columns = lst2
    if store_num == '001库':
        df = df.loc[df.store == '001']
    elif store_num == '002电商库':
        df = df.loc[df.store == '002']
    else:
        df = df.loc[(df.store == '001') | (df.store == '002')]

    df['content'] = df['end_ben'] / df['end_jian']
    df = df[df['store'] != '制表人:']
    df = df[df['store'] != '合计：']
    df = df[df['store'].notnull()]
    df = df.iloc[:, 2:]
    df = df[lst3]
    df['bianma01'] = df['class02'].map(dic_class)
    df['bianma02'] = df['class05'].map(dic_class)
    df = df.sort_values(['bianma01','bianma02'])
    df = df.iloc[:,:-2]
    return df

def danjuChuli(df,store_num):
    if store_num == '001':
        df = df.loc[df['仓库编码'] == '001']
    elif store_num == '002':
        df = df.loc[df['仓库编码'] == '002']
    else:
        df = df.loc[(df['仓库编码'] == '001') | (df['仓库编码'] == '002')]

    # return df
    df = df.dropna(how='all', axis=0)
    cols = [i for i in df.columns.to_list() if 'Unnamed' not in i]
    df = df[cols]
    # get maxrows
    for label, ser in df.items():
        for num0, x in enumerate(ser):
            if isinstance(x, str):
                if '合计' in x:
                    index = num0
                    # print(index)
                    break
    df = df.iloc[:index]
    df = df.dropna(how='all', axis=1)
    return df

def addBen(string):
    pattern = r'(?P<num>\d+)本/件'
    regexp1 = re.compile(pattern)
    mat = regexp1.search(string)
    if mat:
        content = int(mat.group('num'))
    else :
        content  = 0
    return content

# first,choice仓库编号

fname0 =  r"F:\a00nutstore\008\zw08\用友报价\产成品库存\收发存汇总表2024-08.xlsx"
df0 = chuli(fname0,'001库')
df0










,class02,class05,code,stock,content,begin_ben,begin_jian,ruku_ben,ruku_jian,chuku_ben,chuku_jian,end_ben,end_jian
1671,账本,纸面账簿,6001,16K总分类账,60.0,13020.0,217.0,60.0,1.00,120.0,2.00,12960.0,216.0
1672,账本,纸面账簿,6003,16K银行存款日记账,60.0,4140.0,69.0,-41.0,-0.68,679.0,11.32,3420.0,57.0
1673,账本,纸面账簿,6005,16K现金日记账,60.0,3720.0,62.0,-1.0,-0.02,1619.0,26.98,2100.0,35.0
1674,账本,纸面账簿,6101,16K总分类账,40.0,4320.0,108.0,-29.0,-0.73,491.0,12.27,3800.0,95.0
1675,账本,纸面账簿,6102,16K现金流水账(备查账),40.0,1120.0,28.0,NaN,NaN,280.0,7.00,840.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1388,外贸,外贸,美国A6铁订本(枫叶)*,美国A6铁订本(枫叶)*,36.0,324.0,9.0,NaN,NaN,NaN,NaN,324.0,9.0
1389,外贸,外贸,美国A6铁订本*,美国A6铁订本*,36.0,252.0,7.0,NaN,NaN,NaN,NaN,252.0,7.0
1390,外贸,外贸,瑞典A460g方格,瑞典A460g方格,40.0,720.0,18.0,NaN,NaN,NaN,NaN,720.0,18.0
1391,外贸,外贸,瑞典A460g线条,瑞典A460g线条,40.0,2240.0,56.0,NaN,NaN,NaN,NaN,2240.0,56.0


In [2]:
df_yunabiao = df0.copy()


In [3]:
fname_ruku = r"F:\a00nutstore\008\zw08\用友报价\产成品库存调整\其他入库单2024-07-30-IC-2024-07-0023.xlsx"

def getTiaozhen(fname):
    df = pd.read_excel(fname,dtype = {'存货编码':"str"}) #,columns = ['存货编码','数量（本）','数量2（件）']
    df = df[['存货编码','数量（本）','数量2（件）']]
    df.columns = ['code','begin_ben','begin_jian']
    df = df.iloc[:-1]
    return df

df_begin_jia = getTiaozhen(fname_ruku)
df_begin_jia
    

,code,begin_ben,begin_jian
0,EXWGA7100P,5,0.01
1,1101,1,0.02
2,16KPP账夹,5,0.02
3,2003,10,0.04
4,5623,20,0.07
...,...,...,...
91,EFLA560K-甜系定制,35136,122.00
92,HB203C彩,74400,186.00
93,TJ305彩,56150,200.54
94,JSCG1630,48720,203.00


In [4]:
fnmae_chuku = r"F:\a00nutstore\008\zw08\用友报价\产成品库存调整\其他出库单2024-07-30-ID-2024-07-0002.xlsx"
df_begin_jian = getTiaozhen(fnmae_chuku)
df_begin_jian

,code,begin_ben,begin_jian
0,1106,2,0.03
1,5017,240,1.00
2,6125,40,1.00
3,DRXBA6100,256,1.00
4,DRXLB5100,96,1.00
5,DRXYB5100,96,1.00
6,DWXB560N,240,2.00
7,DWXB580N,200,2.00
8,EHJCB560,264,2.00
9,EHKNB560,264,2.00


In [5]:
df_begin_jian['begin_ben'] = df_begin_jian['begin_ben']*-1
df_begin_jian['begin_jian'] = df_begin_jian['begin_jian']*-1
df_begin_jian

,code,begin_ben,begin_jian
0,1106,-2,-0.03
1,5017,-240,-1.00
2,6125,-40,-1.00
3,DRXBA6100,-256,-1.00
4,DRXLB5100,-96,-1.00
5,DRXYB5100,-96,-1.00
6,DWXB560N,-240,-2.00
7,DWXB580N,-200,-2.00
8,EHJCB560,-264,-2.00
9,EHKNB560,-264,-2.00


In [6]:
df_ruku_jian = df_begin_jia.copy()
df_ruku_jian.columns = ['code','ruku_ben','ruku_jian']
df_ruku_jian['ruku_ben'] = df_ruku_jian['ruku_ben']*-1
df_ruku_jian['ruku_jian'] = df_ruku_jian['ruku_jian']*-1
df_ruku_jian

,code,ruku_ben,ruku_jian
0,EXWGA7100P,-5,-0.01
1,1101,-1,-0.02
2,16KPP账夹,-5,-0.02
3,2003,-10,-0.04
4,5623,-20,-0.07
...,...,...,...
91,EFLA560K-甜系定制,-35136,-122.00
92,HB203C彩,-74400,-186.00
93,TJ305彩,-56150,-200.54
94,JSCG1630,-48720,-203.00


In [7]:
df_chuku_jian = df_begin_jian.copy()
df_chuku_jian.columns = ['code','chuku_ben','chuku_jian']


In [8]:
df_chuku_jian

,code,chuku_ben,chuku_jian
0,1106,-2,-0.03
1,5017,-240,-1.00
2,6125,-40,-1.00
3,DRXBA6100,-256,-1.00
4,DRXLB5100,-96,-1.00
5,DRXYB5100,-96,-1.00
6,DWXB560N,-240,-2.00
7,DWXB580N,-200,-2.00
8,EHJCB560,-264,-2.00
9,EHKNB560,-264,-2.00


In [9]:
result = pd.concat([df0,df_begin_jia,df_begin_jian,df_ruku_jian,df_chuku_jian])
result

,class02,class05,code,stock,content,begin_ben,begin_jian,ruku_ben,ruku_jian,chuku_ben,chuku_jian,end_ben,end_jian
1671,账本,纸面账簿,6001,16K总分类账,60.0,13020.0,217.0,60.0,1.00,120.0,2.00,12960.0,216.0
1672,账本,纸面账簿,6003,16K银行存款日记账,60.0,4140.0,69.0,-41.0,-0.68,679.0,11.32,3420.0,57.0
1673,账本,纸面账簿,6005,16K现金日记账,60.0,3720.0,62.0,-1.0,-0.02,1619.0,26.98,2100.0,35.0
1674,账本,纸面账簿,6101,16K总分类账,40.0,4320.0,108.0,-29.0,-0.73,491.0,12.27,3800.0,95.0
1675,账本,纸面账簿,6102,16K现金流水账(备查账),40.0,1120.0,28.0,NaN,NaN,280.0,7.00,840.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,NaN,NaN,NFSX1660NH（双竖线）-1,NaN,NaN,NaN,NaN,NaN,NaN,-160.0,-1.00,NaN,NaN
28,NaN,NaN,NFY1660NH（双色）-1,NaN,NaN,NaN,NaN,NaN,NaN,-160.0,-1.00,NaN,NaN
29,NaN,NaN,NFZW1660NH（300格）-1,NaN,NaN,NaN,NaN,NaN,NaN,-160.0,-1.00,NaN,NaN
30,NaN,NaN,NFZW1660NH（400格）-1,NaN,NaN,NaN,NaN,NaN,NaN,-160.0,-1.00,NaN,NaN


In [10]:
result.to_excel('调期初1.xlsx',index = False)

C:\Users\redda\AppData\Local\Temp\ipykernel_16908\2912535842.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  result.to_excel('调期初1.xlsx',index = False)


In [12]:
huizhong =result.pivot_table(index = 'code',fill_value=0)
huizhong


TypeError: agg function failed [how->mean,dtype->object]

In [13]:
df_fname_yuanqichu = r"F:\a00nutstore\008\zw08\用友报价\产成品库存\收发存汇总表0725账面.xlsx"
df_yuan_qichu = chuli(df_fname_yuanqichu,'001库')
df_yuan_qichu

,class02,class05,code,stock,content,begin_ben,begin_jian,ruku_ben,ruku_jian,chuku_ben,chuku_jian,end_ben,end_jian
1449,账本,纸面账簿,6001,16K总分类账,60.0,13140.0,219.0,NaN,NaN,120.0,2.0,13020.0,217.0
1450,账本,纸面账簿,6003,16K银行存款日记账,60.0,4380.0,73.0,NaN,NaN,240.0,4.0,4140.0,69.0
1451,账本,纸面账簿,6005,16K现金日记账,60.0,4140.0,69.0,NaN,NaN,420.0,7.0,3720.0,62.0
1452,账本,纸面账簿,6101,16K总分类账,40.0,4520.0,113.0,NaN,NaN,200.0,5.0,4320.0,108.0
1453,账本,纸面账簿,6102,16K现金流水账(备查账),40.0,1120.0,28.0,NaN,NaN,NaN,NaN,1120.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,外贸,外贸,美国A6铁订本(枫叶)*,美国A6铁订本(枫叶)*,36.0,324.0,9.0,NaN,NaN,NaN,NaN,324.0,9.0
1216,外贸,外贸,美国A6铁订本*,美国A6铁订本*,36.0,252.0,7.0,NaN,NaN,NaN,NaN,252.0,7.0
1217,外贸,外贸,瑞典A460g方格,瑞典A460g方格,40.0,720.0,18.0,NaN,NaN,NaN,NaN,720.0,18.0
1218,外贸,外贸,瑞典A460g线条,瑞典A460g线条,40.0,2240.0,56.0,NaN,NaN,NaN,NaN,2240.0,56.0


In [15]:
df_yuan_qichu = df_yuan_qichu[['code','end_ben','end_jian']]
df_yuan_qichu

,code,end_ben,end_jian
1449,6001,13020.0,217.0
1450,6003,4140.0,69.0
1451,6005,3720.0,62.0
1452,6101,4320.0,108.0
1453,6102,1120.0,28.0
...,...,...,...
1215,美国A6铁订本(枫叶)*,324.0,9.0
1216,美国A6铁订本*,252.0,7.0
1217,瑞典A460g方格,720.0,18.0
1218,瑞典A460g线条,2240.0,56.0


In [16]:
df_yunabiao1 = df_yunabiao.copy()
df_yunabiao1 = df_yunabiao1[['code','begin_ben','begin_jian']]
df_yunabiao1

,code,begin_ben,begin_jian
1671,6001,13020.0,217.0
1672,6003,4140.0,69.0
1673,6005,3720.0,62.0
1674,6101,4320.0,108.0
1675,6102,1120.0,28.0
...,...,...,...
1388,美国A6铁订本(枫叶)*,324.0,9.0
1389,美国A6铁订本*,252.0,7.0
1390,瑞典A460g方格,720.0,18.0
1391,瑞典A460g线条,2240.0,56.0


In [17]:
df_diff = pd.merge(df_yuan_qichu,df_yunabiao1,how = 'outer')
df_diff


,code,end_ben,end_jian,begin_ben,begin_jian
0,1101,2219.0,36.98,2219.0,36.98
1,1103,894.0,14.90,894.0,14.90
2,1105,900.0,15.00,900.0,15.00
3,1106,6902.0,115.03,6902.0,115.03
4,1107,1140.0,19.00,1140.0,19.00
...,...,...,...,...,...
1777,瑞典A460g线条,2240.0,56.00,2240.0,56.00
1778,瑞典A470g线条,40.0,1.00,40.0,1.00
1779,美国A6铁订本(枫叶)*,324.0,9.00,324.0,9.00
1780,美国A6铁订本*,252.0,7.00,252.0,7.00


In [18]:
df_diff = df_diff.fillna(0)
df_diff

,code,end_ben,end_jian,begin_ben,begin_jian
0,1101,2219.0,36.98,2219.0,36.98
1,1103,894.0,14.90,894.0,14.90
2,1105,900.0,15.00,900.0,15.00
3,1106,6902.0,115.03,6902.0,115.03
4,1107,1140.0,19.00,1140.0,19.00
...,...,...,...,...,...
1777,瑞典A460g线条,2240.0,56.00,2240.0,56.00
1778,瑞典A470g线条,40.0,1.00,40.0,1.00
1779,美国A6铁订本(枫叶)*,324.0,9.00,324.0,9.00
1780,美国A6铁订本*,252.0,7.00,252.0,7.00


In [19]:
df_diff['diff_ben'] = df_diff['end_ben'] - df_diff['begin_ben']
df_diff['diff_jian'] = df_diff['end_jian'] - df_diff['begin_jian']
df_diff

,code,end_ben,end_jian,begin_ben,begin_jian,diff_ben,diff_jian
0,1101,2219.0,36.98,2219.0,36.98,0.0,0.0
1,1103,894.0,14.90,894.0,14.90,0.0,0.0
2,1105,900.0,15.00,900.0,15.00,0.0,0.0
3,1106,6902.0,115.03,6902.0,115.03,0.0,0.0
4,1107,1140.0,19.00,1140.0,19.00,0.0,0.0
...,...,...,...,...,...,...,...
1777,瑞典A460g线条,2240.0,56.00,2240.0,56.00,0.0,0.0
1778,瑞典A470g线条,40.0,1.00,40.0,1.00,0.0,0.0
1779,美国A6铁订本(枫叶)*,324.0,9.00,324.0,9.00,0.0,0.0
1780,美国A6铁订本*,252.0,7.00,252.0,7.00,0.0,0.0


In [20]:
df_diff.to_excel('期初差异变动.xlsx')


C:\Users\redda\AppData\Local\Temp\ipykernel_16908\3886412154.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_diff.to_excel('期初差异变动.xlsx')
